Exercise: Functional overview
--------------------------------------------

For cri-o or containerd we don't have `docker` CLI, so we use `kubectl` from Kubernetes.

Get the container image if it doesn't already exist and start it.

In [ ]:
%%bash
kubectl run webshop --image registry.gitlab.com/ch-mc-b/autoshop/shop:2.0.0 --image-pull-policy="IfNotPresent" \
                        --restart=Never --labels="name=hello,instance=first"

The smallest unit in Kubernetes is a [Pod](https://kubernetes.io/docs/concepts/workloads/pods/). Flattens a group of containers.

We can display the pod (with the container):

In [ ]:
! kubectl get pod/webshop 

With Kubernetes, the container is started in the background, which is why we don't see any output.

In order to see this, we have to unlock the port of the container to the outside.

This is done using:

In [ ]:
! kubectl expose pod/webshop --type="LoadBalancer" --port 80 

Since we don't have a load balancer, we have to prepare the IP of the cluster and the mapped port as a URL using a small shell script.

In [ ]:
! echo "http://"$(cat ~/work/server-ip)":$(kubectl get service webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/"

***

For test purposes we start the container, again under a different name

In [ ]:
! kubectl run webshop1 --image registry.gitlab.com/ch-mc-b/autoshop/shop:2.0.0 --image-pull-policy="IfNotPresent" --restart=Never  --labels="name=hello,instance=second"
! kubectl expose pod/webshop1 --type="LoadBalancer" --port 80
! echo "http://"$(cat ~/work/server-ip)":$(kubectl get service webshop1 -o=jsonpath='{ .spec.ports[0].nodePort }')/"

We now have two containers (in one pod each) running, but still one image.

In [ ]:
! kubectl get pods,services -l name=hello -o wide

In [ ]:
! kubectl get pods -l name=hello -o jsonpath="{..image}" | tr -s '[[:space:]]' '\n' | sed 's/:latest//g' | sort | uniq -c

***

Clean up

In [ ]:
! kubectl delete pod/webshop
! kubectl delete service/webshop
! kubectl delete pod/webshop1
! kubectl delete service/webshop1

The containers (pods) have been terminated, the image still exists.

**Note**: to display the image, run `microk8s ctr image ls` in the VM.

In [ ]:
! kubectl get pods -l name=hello